<a href="https://colab.research.google.com/github/DarkLight23/Music_Generation/blob/main/music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music_genre Classification

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import pathlib
import librosa
import librosa.display
import IPython.display
import csv
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD

import tensorflow as tf

## Collection of DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for folder in os.listdir(f'/content/drive/My Drive/Music/genres/'):
    print(folder)
    pathlib.Path(f'Spec_img/{folder}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'/content/drive/My Drive/Music/genres/{folder}'):
        print(filename)
        songname = f'/content/drive/My Drive/Music/genres/{folder}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        print(y.shape)
        print(sr)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'Spec_img/{folder}/{filename[:-3].replace(".", "_")}.png')
        plt.clf()    

## Generating Mel-Spectrogram 

In [ ]:
header = "music_genres chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate"
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
'''Creating CSV file of each song WIth RMSE,Chroma Stft,Spectral centroid,
Spectral Bandwidth,Zero Cross Rate,MFCC'''
spectral_data = open('Spec_data.csv', 'w', newline='')
with spectral_data:
    writer = csv.writer(spectral_data)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'/content/drive/My Drive/Music/genres/{g}'):
        songname = f'/content/drive/My Drive/Music/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rmse(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        spectral_data = open('Spec_data.csv', 'a', newline='')
        # with file:
        writer = csv.writer(spectral_data)
        writer.writerow(to_append.split())


## Spliting DATA into Train and Test

In [ ]:
!pip install split-folders tqdm

import splitfolders
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio('/content/Spec_img', output="/content/Gtv", seed=1337, ratio=(.80, .20)) # default values



# image augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale = 1./255, # rescale all pixel values from 0-255, so aftre this step all our pixel values are in range (0,1)
        shear_range = 0.2, #to apply some random tranfromations
        zoom_range = 0.2, #to apply zoom
        horizontal_flip = True) # image will be flipper horiz
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# # ImageDataGenerator accepts the original data, randomly transforms it, and returns only the new, transformed data.
training_set = train_datagen.flow_from_directory(
        '/content/Gtv/train',
        target_size = (64,64),
        batch_size = 32,
        class_mode='categorical',
        shuffle = False)
test_set = test_datagen.flow_from_directory(
        '/content/Gtv/val',
        target_size = (64,64,
        batch_size = 32,
        class_mode='categorical',
        shuffle = False )



Found 800 images belonging to 11 classes.
Found 200 images belonging to 11 classes.


## Model used

In [ ]:
from tensorflow.keras.applications import Xception

In [ ]:
base_model=Xception(include_top=False,input_shape=(300,300,3))



# Create new model on top
inputs = tf.keras.Input(shape=(300,300,3))

x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)  # 1st improove Regularize with dropout
x = tf.keras.layers.Dense(512,activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x) # 2st improove Regularize with dropout
x = tf.keras.layers.Dense(64,activation='relu')(x)
outputs = tf.keras.layers.Dense(11,activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, 10, 10, 2048)      20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

## Model Traing and Testing (Using Optimizer = "SGD","ADAM")

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['acc'])
warmup = model.fit(training_set, validation_data=test_set,epochs=100)

Epoch 1/100
25/25 [==============================] - 34s 1s/step - loss: 2.4666 - acc: 0.0575 - val_loss: 2.3824 - val_acc: 0.1000
Epoch 2/100
25/25 [==============================] - 32s 1s/step - loss: 2.3976 - acc: 0.0875 - val_loss: 2.3722 - val_acc: 0.1400
Epoch 3/100
25/25 [==============================] - 32s 1s/step - loss: 2.3908 - acc: 0.0975 - val_loss: 2.3522 - val_acc: 0.1500
Epoch 4/100
25/25 [==============================] - 33s 1s/step - loss: 2.3857 - acc: 0.0600 - val_loss: 2.3247 - val_acc: 0.1700
Epoch 5/100
25/25 [==============================] - 32s 1s/step - loss: 2.3663 - acc: 0.0825 - val_loss: 2.3135 - val_acc: 0.1050
Epoch 6/100
25/25 [==============================] - 32s 1s/step - loss: 2.3601 - acc: 0.0962 - val_loss: 2.2864 - val_acc: 0.2450
Epoch 7/100
25/25 [==============================] - 33s 1s/step - loss: 2.3437 - acc: 0.1325 - val_loss: 2.2686 - val_acc: 0.2150
Epoch 8/100
25/25 [==============================] - 33s 1s/step - loss: 2.3152 - a

In [ ]:
 model.evaluate(test_set)

7/7 [==============================] - 1s 200ms/step - loss: 1.4519 - acc: 0.5700


[1.451855182647705, 0.5699999928474426]

In [ ]:
base_model.trainable = True
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, 10, 10, 2048)      20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-5)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['acc'])
final =  model.fit(training_set, validation_data=test_set,epochs=100)

Epoch 1/100
25/25 [==============================] - 34s 1s/step - loss: 0.4570 - acc: 0.8438 - val_loss: 1.3438 - val_acc: 0.6550
Epoch 2/100
25/25 [==============================] - 33s 1s/step - loss: 0.2991 - acc: 0.9038 - val_loss: 1.2998 - val_acc: 0.6300
Epoch 3/100
25/25 [==============================] - 32s 1s/step - loss: 0.2597 - acc: 0.9137 - val_loss: 1.4204 - val_acc: 0.6450
Epoch 4/100
25/25 [==============================] - 33s 1s/step - loss: 0.2486 - acc: 0.9087 - val_loss: 1.5195 - val_acc: 0.6300
Epoch 5/100
25/25 [==============================] - 33s 1s/step - loss: 0.2215 - acc: 0.9312 - val_loss: 1.5224 - val_acc: 0.6400
Epoch 6/100
25/25 [==============================] - 33s 1s/step - loss: 0.2004 - acc: 0.9275 - val_loss: 1.5647 - val_acc: 0.6350
Epoch 7/100
25/25 [==============================] - 33s 1s/step - loss: 0.1738 - acc: 0.9388 - val_loss: 1.6063 - val_acc: 0.6750
Epoch 8/100
25/25 [==============================] - 33s 1s/step - loss: 0.1646 - a

In [ ]:
test_set.reset()
pred = model.predict_generator(test_set, steps = 20, verbose = 5)

In [ ]:
 model.evaluate(test_set)

### Ploting Accuracy

In [ ]:
plt.plot(range(0,100), warmup.history['acc'], label='Warmup Accuracy (training data)')
plt.plot(range(0,100), warmup.history['val_acc'], label='Warmup Accuracy (validation data)')

plt.plot(range(0,100), final.history['acc'], label='Final Accuracy (training data)')
plt.plot(range(0,100), final.history['val_acc'], label='Final Accuracy (validation data)')

plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
#plt.legend(loc="upper left")
plt.show()

## Saving Output Data into CSV file. 

# New Section

In [ ]:
# nRowsRead = 1000
# Spec_data=pd.read_csv("/content/Spec_data.csv",delimiter=',', nrows = nRowsRead)
# # Spec_data.describe()
# nRow, nCol = Spec_data.shape
# print(f'There are {nRow} rows and {nCol} columns')

# Spec_data.hist()
# scatter_matrix(Spec_data, alpha=0.2, figsize=(20, 20), diagonal='kde')

In [ ]:
# predicted_class_indices = np.argmax(pred,axis=1)

# labels = (training_set.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# predictions = [labels[k] for k in predicted_class_indices]
# predictions = predictions[:200]
# filenames=test_set.filenames

In [ ]:
#  print(len(test_set), len(predictions))

In [ ]:
# results = pd.DataFrame({'Filename':filenames,'Predictions':predictions})
# results.to_csv('prediction_results.csv',index = False)